In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from datasets import Dataset
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments,
)

# --- Step 1. Load dataset ---
df = pd.read_csv("/content/masterList.csv")
df = df.dropna(subset=["text", "label"])
df["label"] = df["label"].astype(int)   # force integer labels

dataset = Dataset.from_pandas(df[["text", "label"]])

# --- Step 2. Train/validation split ---
dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

In [ ]:
# --- Step 3. Tokenizer ---
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

def tokenize_function(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=256)

In [ ]:
train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset  = eval_dataset.map(tokenize_function, batched=True)

# Force label dtype to int for both splits
train_dataset = train_dataset.map(lambda x: {"label": int(x["label"])})
eval_dataset  = eval_dataset.map(lambda x: {"label": int(x["label"])})

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
eval_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

In [ ]:
# --- Step 4. Class weights (moderate) ---
class_weights = torch.tensor([1.0, 1.3, 1.4, 1.5], dtype=torch.float).to("cuda")

# Weighted loss
loss_fn = nn.CrossEntropyLoss(weight=class_weights)

In [ ]:
# --- Step 5. Load Model ---
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=4
).to("cuda")

In [ ]:
# --- Step 7. Custom Trainer ---
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels").to(torch.long)
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss = loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [ ]:
# --- Step 8. Training arguments ---
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    report_to="none",     # <--- disables wandb
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=6,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="f1_weighted",
    greater_is_better=True,
)

# --- Step 9. Trainer ---
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    processing_class=tokenizer,  # future-proof
    compute_metrics=compute_metrics,
)

# --- Step 10. Train ---
trainer.train()

In [ ]:
# --- Step 11. Evaluate ---
predictions = trainer.predict(eval_dataset)
y_true = predictions.label_ids
y_pred = np.argmax(predictions.predictions, axis=1)

print(classification_report(
    y_true, y_pred,
    target_names=["Valid (0)", "Spam (1)", "LowQuality (2)", "Rant (3)"]
))

In [ ]:
# --- Step 12. Confusion Matrix ---
cm = confusion_matrix(y_true, y_pred, labels=[0,1,2,3])
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Valid (0)", "Spam (1)", "LowQuality (2)", "Rant (3)"],
            yticklabels=["Valid (0)", "Spam (1)", "LowQuality (2)", "Rant (3)"])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()
